In [87]:
# First cell - Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from scipy import stats
import requests
import statsmodels as sm
import kaleido
import os
from dotenv import load_dotenv

In [62]:
# Second Cell - Data Extraction/Load
load_dotenv()

# Get API key from environment
API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'

def get_fred_series(series_id, observation_start=None):
    """
    Fetch data series from FRED API
    
    Parameters:
    series_id (str): FRED series identifier
    observation_start (str): Start date in YYYY-MM-DD format
    
    Returns:
    pandas.DataFrame: Time series data
    """
    # Build API URL
    url = f"{BASE_URL}/series/observations"
    
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': observation_start if observation_start else '1976-01-01'
    }
    
    # Make API request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    return df.set_index('date')['value'] #set date index so pandas automatically knows the date (i.e. resampling)

# Get our series
yield_spread = get_fred_series('T10Y2Y')
gdp = get_fred_series('GDPC1')
fed_funds = get_fred_series('DFF')
unemployment = get_fred_series('UNRATE')
option_adjusted_spread = get_fred_series('BAMLH0A0HYM2')
delinquency_rate_credit = get_fred_series('DRCCLACBS')
delinquency_rate_loans = get_fred_series('DRBLACBS')
# Create main dataframe
df = pd.DataFrame({
    'yield_spread': yield_spread,
    'gdp': gdp,
    'fed_funds':fed_funds, 
    'unemployment': unemployment,
    'option_adjusted_spread': option_adjusted_spread,
    'delinquency_rate_credit_cards': delinquency_rate_credit,
    'delinquency_rate_loans': delinquency_rate_loans
})

 



In [ ]:
# Third Cell - Data Transformation

# Calculate GDP growth rate (annualized)
df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100

#Fill-down 
df['delinquency_rate_credit_cards'] = df['delinquency_rate_credit_cards'].ffill()
df['delinquency_rate_loans'] = df['delinquency_rate_loans'].ffill()

# Get end of month value for option adjusted spread
df['quarterly_spread'] = df.groupby(df.index.to_period('Q'))['option_adjusted_spread'].transform('mean')

# Fill down (forward fill)
df['quarterly_spread'] = df['quarterly_spread'].ffill()

df = df.reset_index()
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df['quarter'] = df.index.quarter.astype(str) + "Q" + df.index.year.astype(str).str[-2:]
df['period'] = np.where((df.index >= '2001-01-01') & (df.index <= '2001-12-31'), 'Dot Com',
                 np.where((df.index >= '2007-10-01') & (df.index <= '2009-06-30'), 'Great Recession',
                 np.where((df.index >= '2020-01-01') & (df.index <= '2020-06-30'), 'COVID',
                 'Expansion')))
df.to_excel('stats.xlsx')
display(df)

C:\Users\mrasm\AppData\Local\Temp\ipykernel_3752\3652860053.py:4: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,gdp_growth,delinquency_rate_clean,quarterly_spread,quarter,period
date,,,,,,,,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-02,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-03,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-04,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-05,NaN,NaN,5.29,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1Q76,Expansion
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-04,0.14,NaN,4.83,NaN,2.87,3.25,1.13,0.0,3.25,2.908214,4Q24,Expansion
2024-11-05,0.07,NaN,4.83,NaN,2.86,3.25,1.13,0.0,3.25,2.908214,4Q24,Expansion
2024-11-06,0.15,NaN,4.83,NaN,2.74,3.25,1.13,0.0,3.25,2.908214,4Q24,Expansion


In [93]:
# Fourth Cell - Data Analysis (Two Variables)

# Create new dataframe with only necessary data
df_viz = df.reset_index().copy()
df_viz = df_viz[df_viz['date'] >= '1996-12-31']
df_viz = df_viz[['quarter', 'period', 'quarterly_spread', 'delinquency_rate_loans']]
df_viz = df_viz.drop_duplicates()
display(df_viz)
spread_loan_correlation = df_viz[['quarterly_spread', 'delinquency_rate_loans']].corr()
print("Correlation between Option-Adjusted Spread and delinquency rates:")
print(spread_loan_correlation)
fig = px.scatter(df_viz,
                x='quarterly_spread',
                y='delinquency_rate_loans',
                color='period',
                title=f'Corporate Loan Delinquency Rate vs Option-Adjusted Spread (Correlation: {spread_loan_correlation.iloc[0,1]:.3f})',
                labels={'quarterly_spread': 'Option-Adjusted Spread',
                       'delinquency_rate_loans': 'Delinquency Rate (%)',
                       'period': 'Economic Period'},
                trendline="ols",
                color_discrete_map={
                    'Expansion': 'blue',
                    'Dot Com': 'red',
                    'Great Recession': 'purple',
                    'COVID': 'orange'
                })

fig.show()

,quarter,period,quarterly_spread,delinquency_rate_loans
7670,4Q96,Expansion,3.130000,1.95
7671,1Q97,Expansion,2.897377,1.90
7761,2Q97,Expansion,2.747231,1.81
7852,3Q97,Expansion,2.620462,1.72
7944,4Q97,Expansion,2.780606,1.70
...,...,...,...,...
17440,4Q23,Expansion,3.978000,1.03
17532,1Q24,Expansion,3.365000,1.12
17623,2Q24,Expansion,3.180152,1.13
17714,3Q24,Expansion,3.259851,1.13


Correlation between Option-Adjusted Spread and delinquency rates:
                        quarterly_spread  delinquency_rate_loans
quarterly_spread                1.000000                0.608238
delinquency_rate_loans          0.608238                1.000000


In [84]:
# Fifth Cell - Data Visualization

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add spreads
fig.add_trace(
    go.Scatter(x=df_viz['quarter'], y=df_viz['quarterly_spread'], name="Option-Adjusted Spread %"),
    secondary_y=False
)

# Add delinquency
fig.add_trace(
    go.Scatter(x=df_viz['quarter'], y=df_viz['delinquency_rate_loans'], name="Delinquency Rate Loan %"),
    secondary_y=True
)

# Update layout 
fig.update_layout(
    title='Option-Adjusted Spread vs Corporate Loan Delinquency'
)

fig.update_yaxes(title_text="Option-Adjusted Spread (%)", secondary_y=False)
fig.update_yaxes(title_text="Delinquency Rate (%)", secondary_y=True)

fig.show()
pio.write_image(fig, "data_viz.pdf", format="pdf", engine="kaleido")

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_viz.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_viz[col] for col in df_viz.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()